In [1]:
!pip install keras-tuner --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.3 MB/s eta 0:00:00


In [2]:
#https://github.com/Lamaratamleh/lamar_atamleh.git
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

# Load MNIST dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Extract dataset info
num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples

print(f"Number of classes: {num_of_classes}")
print(f"Image shape: {image_shape}")
print(f"Training samples: {train_size}")
print(f"Testing samples: {test_size}")

# Normalize and preprocess datasets
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    image = tf.reshape(image, [-1])  # Flatten the image to 1D
    label = tf.one_hot(label, depth=num_of_classes)  # One-hot encode labels
    return image, label

ds_train = ds_train.map(preprocess).cache().shuffle(10000).batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(128).cache().prefetch(tf.data.AUTOTUNE)

# Define the ID and parse the number of neurons for each fully connected layer
id_number = "212760482"
neurons = [
    int(id_number[-2:]),    # 82
    int(id_number[-4:-2]),  # 04
    int(id_number[-6:-4]),  # 76
    int(id_number[-8:-6])   # 12
]
print(f"Neurons in each layer: {neurons}")

# Build the model
model = models.Sequential()

# Input layer
model.add(layers.Input(shape=(28 * 28,)))

# Fully connected layers
model.add(layers.Dense(neurons[0]))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.1))

model.add(layers.Dense(neurons[1]))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.1))

model.add(layers.Dense(neurons[2]))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.1))

model.add(layers.Dense(neurons[3]))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.2))

# Output layer
model.add(layers.Dense(num_of_classes, activation="softmax"))

# Compile the model
model.compile(
     optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test
)

# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Number of classes: 10
Image shape: (28, 28, 1)
Training samples: 60000
Testing samples: 10000
Neurons in each layer: [82, 4, 76, 12]
Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 30ms/step - accuracy: 0.5887 - loss: 1.2187 - val_accuracy: 0.9209 - val_loss: 0.3135
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7951 - loss: 0.6461 - val_accuracy: 0.9399 - val_loss: 0.2526
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8281 - loss: 0.5615 - val_accuracy: 0.9427 - val_loss: 0.2311
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8502 - loss: 0.5037 - val_accuracy: 0.9419 - val_loss: 0.2164
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8553 - loss: 0.4763 - val_accuracy: 0.9497 - val_loss: 0.1978
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8638 - loss: 0.4523 - val_accuracy: 0.93

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
import tensorflow_datasets as tfds

# Load MNIST dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Extract dataset info
num_of_classes = ds_info.features['label'].num_classes

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    image = tf.reshape(image, [-1])  # Flatten the image to 1D
    label = tf.one_hot(label, depth=num_of_classes)  # One-hot encode labels
    return image, label

ds_train = ds_train.map(preprocess).cache().shuffle(10000).batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(128).cache().prefetch(tf.data.AUTOTUNE)

# Define the ID and parse the number of neurons for each fully connected layer
id_number = "212760482"
neurons = [
    int(id_number[-2:]),    # 82
    int(id_number[-4:-2]),  # 04
    int(id_number[-6:-4]),  # 76
    int(id_number[-8:-6])   # 12
]
print(f"Neurons in each layer: {neurons}")

# Define the model builder function for KerasTuner
def model_builder(hp):
    model = keras.Sequential()

    # Input layer
    model.add(layers.Input(shape=(28 * 28,)))

    # Fully connected layers with tunable dropout
    for i, n in enumerate(neurons):
        model.add(layers.Dense(n))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation("relu"))
        model.add(layers.Dropout(hp.Float(f"dropout_{i}", min_value=0.1, max_value=0.5, step=0.1)))

    # Output layer
    model.add(layers.Dense(num_of_classes, activation="softmax"))

    # Compile the model with a tunable learning rate
    learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Initialize the tuner

tuner = kt.Hyperband(
    model_builder,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="my_dir",
    project_name="mnist_tuning"
)

# Search for the best hyperparameters
tuner.search(ds_train, epochs=10, validation_data=ds_test)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print(f"Best dropout rates: {[best_hps.get(f'dropout_{i}') for i in range(len(neurons))]}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

# Build the best model and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(ds_train, epochs=10, validation_data=ds_test)

# Evaluate the best model
loss, accuracy = model.evaluate(ds_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

Trial 30 Complete [00h 00m 36s]
val_accuracy: 0.7372999787330627

Best val_accuracy So Far: 0.9517999887466431
Total elapsed time: 00h 14m 02s
Best dropout rates: [0.2, 0.1, 0.30000000000000004, 0.1]
Best learning rate: 0.001
Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.3876 - loss: 1.7359 - val_accuracy: 0.8060 - val_loss: 0.8101
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.6724 - loss: 0.9638 - val_accuracy: 0.9184 - val_loss: 0.4106
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7748 - loss: 0.7734 - val_accuracy: 0.9311 - val_loss: 0.3339
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7998 - loss: 0.6977 - val_accuracy: 0.9348 - val_loss: 0.3153
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8162 - loss: 0.6524 - val_accuracy: 0.9376 - val_loss: 0.2943
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8237 - loss: 0.6121 - val_accuracy: 0.9417 - val_loss: 0.2667
Epo